In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from bcb import sgs
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

#### Dados gerais - IPCA mensal e acumulado 

In [ ]:
#IPCA - variação mensal
ipca = sgs.get({'IPCA':433},start = "2013-02-02")
ipca.reset_index(inplace=True)

df_ipca = pd.DataFrame(ipca)
print(df_ipca)

#### Pegando Meta e limites - IPCA  

In [ ]:
df_meta = sgs.get({'Meta':13521}, start = '2013-02-02')

df_meta = df_meta.reset_index()

data_mensal = [] 

for _, row in df_meta.iterrows():
    ano = row['Date']
    meta = row['Meta']
    
    for mes in range(1, 13):  
        data_mensal.append({
            'Date': pd.Timestamp(f'{ano}-{mes:02d}-01'),
            'Meta': meta})

df_meta_mensal = pd.DataFrame(data_mensal)
df_meta_mensal = df_meta_mensal.drop(columns=['Date'])

df_meta_mensal['Limite superior'] = df_meta_mensal['Meta'] +1.5
df_meta_mensal['Limite inferior'] = df_meta_mensal['Meta'] -1.5

print(df_meta_mensal.tail(50))

In [ ]:
#IPCA acumulado 12 meses - Janela móvel 
ipca_12m = ipca['IPCA'].rolling(12).apply(lambda x: (1+x/100).prod()-1).dropna()*100
ipca_12m = round(ipca_12m,2)

merged_ipca = pd.merge(ipca, ipca_12m, left_index=True, right_index=True)
merged_ipca = merged_ipca.rename(columns={'IPCA_x': 'IPCA Mensal', "IPCA_y":'IPCA var 12m'})

inflacao = pd.merge(merged_ipca, df_meta_mensal, left_index=True, right_index=True)

print(inflacao.tail(10))


In [ ]:
#Gráfico IPCA 12 meses 
plt.figure(figsize=(10, 6))

plt.rcParams['font.family'] = 'Roc Grotesk' 
plt.title('Variação do IPCA - Acumulado 12 meses', fontsize=16)

sns.lineplot(data=merged_ipca, x='Date', y='IPCA var 12m', color='limegreen', linewidth=2.5)
plt.plot(inflacao['Date'], inflacao['Limite superior'], linestyle='--', color='gray')
plt.plot(inflacao['Date'], inflacao['Limite inferior'], linestyle='--', color='gray')

plt.grid(axis='y', color='gray', alpha=0.25)

plt.gca().xaxis.set_major_formatter(DateFormatter('%b/%y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 7]))     
plt.gcf().autofmt_xdate()

plt.xlim(merged_ipca['Date'].min(), merged_ipca['Date'].max())

plt.ylabel('Inflação (%)', fontsize=14)
plt.xlabel('Data', fontsize=1)
sns.despine()

plt.show()

#### Criando função que extrai as séries

In [ ]:
#Função que servirá para pegar as séries

def get_series(codigo, date):
    df = sgs.get({codigo:codigo},start =date)

    return df

#### Componentes do IPCA

In [ ]:
#Criar dicionário que armazena as séries abaixo - Componentes do IPCA

data='2013-02-02'

componentes_ipca = {
    'Saúde e Cuidados pessoais': 1641,
    'Alimentação e Bebidas': 1635,
    'Habitação': 1636,
    'Artigos de residência': 1637,  
    'Vestuário': 1638,
    'Transportes': 1639,
    'Despesas pessoais': 1642,
    'Comunicação': 1640,
    'Educação': 1643  
}

df_componentes = pd.DataFrame()

for nome, codigo in componentes_ipca.items():
    df_componentes[nome] = get_series(codigo, data)

#Variação mensal
df_componentes = round(df_componentes,2)

#Convertendo para variação anual
df_componentes_ano = df_componentes.rolling(12).apply(lambda x: (1 + x / 100).prod() - 1).dropna() * 100
df_componentes_ano = round(df_componentes_ano, 2)

print(df_componentes_ano.tail(5))
print(df_componentes.tail(5))

In [ ]:
cores = ['green', 'lime','black','purple','blue','red','orange','gold','gray']

plt.figure(figsize=(12,8))
plt.ylim(-7,25)

sns.set_style('white')  

plt.rcParams['font.family'] = 'Aptos Narrow' 

#Plot da variação anual

for i, coluna in enumerate(df_componentes_ano.columns):
    sns.lineplot(data=df_componentes_ano, x=df_componentes_ano.index, y=coluna, color=cores[i], label=coluna)
    plt.plot(inflacao['Date'], inflacao['Limite superior'], linestyle='--', color='gray')
    plt.plot(inflacao['Date'], inflacao['Limite inferior'], linestyle='--', color='gray')

plt.gca().xaxis.set_major_formatter(DateFormatter('%b/%y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 7]))     
plt.gcf().autofmt_xdate()

plt.xlim(df_componentes_ano.index.min(), df_componentes_ano.index.max())

plt.grid(axis='y', color='gray', alpha=0.25)

plt.title('IPCA - Componentes (var 12m)', fontsize=14)
plt.ylabel('Inflação (%)',fontsize=12)
plt.xlabel('Data', fontsize = 12)
plt.legend(loc='upper left', fontsize='medium',bbox_to_anchor=(0.98,0.98))

sns.despine()

plt.show()

In [ ]:
df_componentes_ano_bar = df_componentes_ano.iloc[-1]
df_componentes_ano_bar_last = df_componentes_ano.iloc[-2]

df_componentes_bar = df_componentes.iloc[-1]
df_componentes_bar_last = df_componentes.iloc[-2]

In [ ]:
nomes = [
    'Saúde e\nCuidados\npessoais',
    'Alimentação\ne Bebidas',
    'Habitação',
    'Artigos de\nresidência',
    'Vestuário',
    'Transportes',
    'Despesas\npessoais',
    'Comunicação',
    'Educação'
]

#PLOT para inflação dos componentes - Ano x Ano

plt.figure(figsize=(10, 6))

plt.rcParams['font.family'] = 'Roc Grotesk' 

bar_width = 0.35

x = np.arange(len(df_componentes_ano_bar))

bars = plt.bar(x, df_componentes_ano_bar.values, width=bar_width, color='limegreen', label='Inflação corrente')

# Adicionando rótulos às barras
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width()/2.0, 
        height + 0.2, 
        f'{height:.2f}', 
        ha='center', 
        va='bottom'
    )

plt.bar(x + 0.25, df_componentes_ano_bar_last.values, width=0.12, color='dimgrey', label='Inflação do mês anterior')
plt.grid(axis='y', color='gray', alpha=0.25)

plt.title('Inflação dos componentes - YoY', fontsize=16)
plt.ylabel('IPCA (var 12m)', fontsize=14)
plt.ylim(-1,10)

plt.xticks(x + 0.225, nomes, rotation=0, fontsize = 10)

plt.legend()

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.show()


#### Núcleos do IPCA

In [ ]:
#Criar dicionário que armazena as séries abaixo - Núcleos do IPCA
nucleos_ipca = {
    'IPCA EX0': 11427,
    'IPCA DP': 16122,
    'IPCA MS': 4466,
    'IPCA EX3': 27839,
    'IPCA P55': 28750, 
}

df_nucleos = pd.DataFrame()
df_nucleos_ano = pd.DataFrame()

for nome, codigo in nucleos_ipca.items():
    df_nucleos[nome] = get_series(codigo, data)

#Variação mensal dos núcleos
df_nucleos = round(df_nucleos,2)

#Convertendo para variação anual
df_nucleos_ano = df_nucleos.rolling(12).apply(lambda x: (1 + x / 100).prod() - 1).dropna() * 100
df_nucleos_ano = round(df_nucleos_ano, 2)

print(df_nucleos.tail(5))
print(df_nucleos_ano.tail(5))

In [ ]:
cores = ['lime','black','green','purple','gray']

#PLOT - Variação dos núcleos - YoY

plt.figure(figsize=(12,8))
plt.ylim(0,12)

sns.set_style('white')  

plt.rcParams['font.family'] = 'Aptos Narrow' 

for i, coluna in enumerate(df_nucleos_ano.columns):
    sns.lineplot(data=df_nucleos_ano, x='Date', y=coluna, color = cores[i],label=coluna, linewidth=1.6)
    plt.plot(inflacao['Date'], inflacao['Limite superior'], linestyle='--', color='gray')
    plt.plot(inflacao['Date'], inflacao['Limite inferior'], linestyle='--', color='gray')

plt.grid(axis='y', color='gray', alpha=0.25)
plt.gca().xaxis.set_major_formatter(DateFormatter('%b/%y'))   
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 7]))     
plt.gcf().autofmt_xdate()
plt.xlim(df_nucleos_ano.min(), df_nucleos_ano.max())

plt.title('Variação do IPCA (YoY) - Núcleos', fontsize=14)
plt.ylabel('Inflação (%)',fontsize=12)
plt.xlabel('Data', fontsize = 12)
plt.legend(loc='upper right', fontsize='medium')

sns.despine()

plt.show()

In [ ]:
df_nucleos_ano['Média']= (df_nucleos_ano['IPCA EX0'] + df_nucleos_ano['IPCA DP'] + df_nucleos_ano['IPCA MS'] + df_nucleos_ano['IPCA EX3'] + df_nucleos_ano['IPCA P55'])/5
mean_nucleos = df_nucleos_ano['Média']
std_nucleos = df_nucleos_ano['Média'].std()

plt.figure(figsize=(12,8))

plt.rcParams['font.family'] = 'Aptos Narrow' 

sns.set_style('white')  

sns.lineplot(data=df_nucleos_ano, x='Date', y='Média',color='limegreen',linewidth=2.5,label='Média 5 núcleos')
sns.lineplot(data=merged_ipca, x='Date', y='IPCA var 12m', color='black',linewidth=2.5,label='IPCA')

plt.plot(inflacao['Date'], inflacao['Limite superior'], linestyle='--', color='gray')
plt.plot(inflacao['Date'], inflacao['Limite inferior'], linestyle='--', color='gray')
plt.grid(axis='y', color='gray', alpha=0.25)

plt.gca().set_yticks(range(int(merged_ipca['IPCA var 12m'].min()), int(merged_ipca['IPCA var 12m'].max()) + 3, 1))
plt.gca().xaxis.set_major_formatter(DateFormatter('%b/%y'))  
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 7]))     
plt.gcf().autofmt_xdate()
plt.xlim(merged_ipca['Date'].min(), merged_ipca['Date'].max())

plt.title('Variação em 12 meses encadeada - IPCA', fontsize=14)
plt.ylabel('Inflação (%)',fontsize=12)
plt.xlabel('Data', fontsize = 12)
sns.despine()

plt.show()


#### Categorias de uso do IPCA

In [ ]:
#Criar dicionário que armazena as séries abaixo - Categorias de uso do IPCA

categorias_ipca = {
    'Não duráveis': 10841,
    'Semiduráveis': 10842,
    'Duráveis': 10843,
    'Serviços': 10844,
}

df_categorias = pd.DataFrame()
df_categorias_ano = pd.DataFrame()

for nome, codigo in categorias_ipca.items():
    df_categorias[nome] = get_series(codigo, data)

#Variação IPCA - mensal
df_categorias = round(df_categorias,2)

#Convertendo variação para anual
df_categorias_ano = df_categorias.rolling(12).apply(lambda x: (1 + x / 100).prod() - 1).dropna() * 100
df_categorias_ano = round(df_categorias_ano, 2)

print(df_categorias.tail(5))
print(df_categorias_ano.tail(5))

In [ ]:
cores = ['green', 'lime','black','purple']

#PLOT - Categorias YoY

plt.figure(figsize=(12,8))
plt.ylim(-3,20)

sns.set_style('white')  

plt.rcParams['font.family'] = 'Aptos Narrow' 

for i, coluna in enumerate(df_categorias_ano.columns):
    sns.lineplot(data=df_categorias_ano, x=df_categorias_ano.index, y=coluna, color=cores[i], linewidth=2, label=coluna)
    plt.plot(inflacao['Date'], inflacao['Limite superior'], linestyle='--', color='gray')
    plt.plot(inflacao['Date'], inflacao['Limite inferior'], linestyle='--', color='gray')

plt.gca().set_yticks(range(int(df_categorias_ano['Semiduráveis'].min()), int(df_categorias_ano['Semiduráveis'].max()) + 3, 1))
plt.grid(axis='y', color='gray', alpha=0.25)

plt.gca().xaxis.set_major_formatter(DateFormatter('%b/%y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 7]))     
plt.gcf().autofmt_xdate()
plt.xlim(df_categorias_ano.index.min(), df_categorias_ano.index.max())


plt.title('IPCA - Categorias (var 12m)', fontsize=14)
plt.ylabel('Inflação (%)',fontsize=12)
plt.xlabel('Data', fontsize = 12)
plt.legend(loc='upper right', fontsize='medium')

sns.despine()

plt.show()

#### Comercialização IPCA

In [ ]:
#Criar dicionário que armazena as séries abaixo - Comercialização

comercializacao_ipca = {
    'Comercializáveis': 4447,
    'Não Comercializáveis': 4448,
}

df_comercializacao = pd.DataFrame()
df_comercializacao_ano = pd.DataFrame()

for nome, codigo in comercializacao_ipca.items():
    df_comercializacao[nome] = get_series(codigo, data)

#IPCA - Variação mensal 
df_comercializacao = round(df_comercializacao,2)

#IPCA - Convertendo para variação anual 
df_comercializacao_ano = df_comercializacao.rolling(12).apply(lambda x: (1 + x / 100).prod() - 1).dropna() * 100
df_comercializacao_ano = round(df_comercializacao_ano, 2)

print(df_comercializacao.tail(5))
print(df_comercializacao_ano.tail(5))

In [ ]:
cores = ['lime','black']

plt.figure(figsize=(12,8))
plt.ylim(-2,15)

plt.rcParams['font.family'] = 'Aptos Narrow' 

sns.set_style('white')  

for i, coluna in enumerate(df_comercializacao_ano.columns):
    sns.lineplot(data=df_comercializacao_ano, x=df_comercializacao_ano.index, y=coluna,color = cores[i],label=coluna,linewidth=2.5)
    plt.plot(inflacao['Date'], inflacao['Limite superior'], linestyle='--', color='gray')
    plt.plot(inflacao['Date'], inflacao['Limite inferior'], linestyle='--', color='gray')


plt.gca().set_yticks(range(int(df_comercializacao_ano['Comercializáveis'].min()-3), int(df_comercializacao_ano['Comercializáveis'].max()) + 3, 1))
plt.grid(axis='y', color='gray', alpha=0.25)

plt.gca().xaxis.set_major_formatter(DateFormatter('%b/%y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 7]))     
plt.gcf().autofmt_xdate()
plt.xlim(df_comercializacao_ano.index.min(), df_comercializacao_ano.index.max())

plt.title('Variação do IPCA - Comercialização (YoY)', fontsize=14)
plt.ylabel('Inflação (%)',fontsize=12)
plt.xlabel('Data', fontsize = 12)
plt.legend(loc='upper right', fontsize='medium')

sns.despine()

plt.show()


#### IPCA - Administrados e Livres

In [ ]:
administrados_ipca = {
    'Administrados': 4449,
    'Total Livres': 11428,
}

df_administrados = pd.DataFrame()
df_administrados_ano = pd.DataFrame()

for nome, codigo in administrados_ipca.items():
    df_administrados[nome] = get_series(codigo, data)

#IPCA - Variação mensal (%)
df_administrados = round(df_administrados,2)

#IPCA - Convertendo para variação anual (%)
df_administrados_ano = df_administrados.rolling(12).apply(lambda x: (1 + x / 100).prod() - 1).dropna() * 100
df_administrados_ano = round(df_administrados_ano, 2)

print(df_administrados.tail(5))
print(df_administrados_ano.tail(5))


In [ ]:
cores = ['lime','black']

plt.figure(figsize=(10,6))

plt.rcParams['font.family'] = 'Roc Grotesk' 

plt.title('Variação do IPCA - Administrados x Livres (YoY)', fontsize=14)

sns.set_style('white')  

for i, coluna in enumerate(df_administrados_ano.columns):
    sns.lineplot(data=df_administrados_ano, x=df_administrados_ano.index, y=coluna,color = cores[i],label=coluna,linewidth=2.5)
    plt.plot(inflacao['Date'], inflacao['Limite superior'], linestyle='--', color='gray')
    plt.plot(inflacao['Date'], inflacao['Limite inferior'], linestyle='--', color='gray')

plt.gca().set_yticks(range(int(df_administrados_ano['Administrados'].min()), int(df_administrados_ano['Administrados'].max()), 1))
plt.grid(axis='y', color='gray', alpha=0.25)
plt.gca().xaxis.set_major_formatter(DateFormatter('%b/%y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 8]))     
plt.gcf().autofmt_xdate()
plt.xlim(df_administrados_ano.index.min(), df_administrados_ano.index.max())

plt.ylabel('Inflação (%)',fontsize=12)
plt.xlabel('Data', fontsize = 12)
plt.legend(loc='upper right', fontsize='medium')

sns.despine()

plt.show()

#### IPCA - Difusão (MM12M)

In [ ]:
df_difusao = pd.DataFrame()
df_difusao_ano = pd.DataFrame()

#Difusão mensal
df_difusao['Difusão'] = get_series(21379,data)
df_difusao = round(df_difusao, 2)

#Convertendo difusão para YoY
df_difusao_ano['Difusão MM12M'] = df_difusao.rolling(12).mean()
df_difusao_ano = round(df_difusao_ano, 2)

print(df_difusao.tail(5))
print(df_difusao_ano.tail(5))

In [ ]:
#Difusão = Mensal

plt.figure(figsize=(12,8))

plt.rcParams['font.family'] = 'Aptos Narrow' 

sns.set_style('white')  

sns.lineplot(data=df_difusao_ano, x='Date', y='Difusão MM12M', color='darkgreen',linewidth=2.5, label = 'MM12M')
sns.lineplot(data=df_difusao, x='Date', y='Difusão', color='lime',linewidth=2.5,label='Difusão')
plt.grid(axis='y', color='gray', alpha=0.25)

plt.gca().xaxis.set_major_formatter(DateFormatter('%b/%y'))
    
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 7]))     
plt.gcf().autofmt_xdate()

plt.xlim(df_difusao_ano.index.min(), df_difusao_ano.index.max())

plt.title('Difusão - IPCA', fontsize=14)
plt.ylabel('Difusão (%)',fontsize=14)
plt.xlabel('Data', fontsize = 14)
sns.despine()

plt.show()


#### Criando função que dessazonaliza os dataframes

In [ ]:
def dessazonalizar(df):
    dessazonalized_df = pd.DataFrame(index=df.index)
    for column in df.columns:
        decomposition = seasonal_decompose(df[column], model='additive', period=12)
        dessazonalized_df[f'{column}'] = df[column] - decomposition.seasonal
    return dessazonalized_df
    

#### Aplicando dessaz para todos os dataframes

In [ ]:
#Dataframes que armazenam a variação da inflação mensal

df_componentes_dessaz = round(dessazonalizar(df_componentes),2)
df_categoria_dessaz = round(dessazonalizar(df_categorias))
df_nucleos_dessaz = round(dessazonalizar(df_nucleos),2)
df_comercializacao_dessaz = round(dessazonalizar(df_comercializacao),2)
df_administrados_dessaz = round(dessazonalizar(df_administrados),2)

In [ ]:
#Dataframes que armazenam a variação acumulada 12 meses da inflação

df_componentes_ano_dessaz = round(dessazonalizar(df_componentes_ano),2)
df_categoria_ano_dessaz = round(dessazonalizar(df_categorias_ano))
df_nucleos_ano_dessaz = round(dessazonalizar(df_nucleos_ano),2)
df_comercializacao_ano_dessaz = round(dessazonalizar(df_comercializacao_ano),2)
df_administrados_ano_dessaz = round(dessazonalizar(df_administrados_ano),2)

print(df_nucleos_ano_dessaz)

#### IPCA x IPCA e Núcleos dessaz

In [ ]:
plt.figure(figsize=(12,8))

plt.rcParams['font.family'] = 'Aptos Narrow' 

sns.set_style('white')  

sns.lineplot(data=df_nucleos_ano_dessaz, x='Date', y='Média',color='purple',linestyle= 'dotted',linewidth=4,label='Núcleo dessaz')
sns.lineplot(data=merged_ipca, x='Date', y='IPCA var 12m',color='limegreen',linewidth=2.5,label='IPCA')
plt.plot(inflacao['Date'], inflacao['Limite superior'], linestyle='--', color='gray')
plt.plot(inflacao['Date'], inflacao['Limite inferior'], linestyle='--', color='gray')
plt.grid(axis='y', color='gray', alpha=0.25)

plt.gca().set_yticks(range(int(merged_ipca['IPCA var 12m'].min()), int(merged_ipca['IPCA var 12m'].max())+2, 1))

plt.gca().xaxis.set_major_formatter(DateFormatter('%b/%y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 7]))     
plt.gcf().autofmt_xdate()
plt.xlim(df_nucleos_ano_dessaz.index.min(), df_nucleos_ano_dessaz.index.max())

plt.title('Variação em 12 meses encadeada - IPCA', fontsize=14)
plt.ylabel('Inflação (%)',fontsize=12)
plt.xlabel('Data', fontsize = 12)
sns.despine()

plt.show()

#### Criando função que calcula média móvel de três meses sobre o dataframe dessazonalizado

In [ ]:
def three_month_moving_average(df):
    moving_averages_df = pd.DataFrame(index=df.index)
    for column in df.columns:
        moving_averages_df[f'{column} MM3M'] = df[column].rolling(window=3).mean()
    return moving_averages_df

#### Calculando a média móvel de 3 meses dessazonalizada para cada Data Frame

In [ ]:
df_componentes_ma = round(three_month_moving_average(df_componentes_dessaz),2)
df_categorias_ma = round(three_month_moving_average(df_categoria_dessaz),2)
df_nucleos_ma = round(three_month_moving_average(df_nucleos_dessaz),2)
df_comercializacao_ma = round(three_month_moving_average(df_comercializacao_dessaz),2)
df_administrados_ma = round(three_month_moving_average(df_administrados_dessaz),2)

In [ ]:
df_componentes_ano_ma = round(three_month_moving_average(df_componentes_ano_dessaz),2)
df_categorias_ano_ma = round(three_month_moving_average(df_categoria_ano_dessaz),2)
df_nucleos_ano_ma = round(three_month_moving_average(df_nucleos_ano_dessaz),2)
df_comercializacao_ano_ma = round(three_month_moving_average(df_comercializacao_ano_dessaz),2)
df_administrados_ano_ma = round(three_month_moving_average(df_administrados_ano_dessaz),2)

#### Gráficos - IPCA Dessaz

In [ ]:
def generate_plots(min, max, df,texto):

    plt.rcParams['font.family'] = 'Aptos Narrow' 

    cores = ['lime','black','indigo','gray','green','blue']

    plt.figure(figsize=(12,8))
    plt.ylim(min,max)

    sns.set_style('white')  

    for i, coluna in enumerate(df.columns):
        sns.lineplot(data=df, x=df.index, y=coluna,color = cores[i],label=coluna,linewidth=2.5)
        plt.plot(inflacao['Date'], inflacao['Limite superior'], linestyle='--', color='gray')
        plt.plot(inflacao['Date'], inflacao['Limite inferior'], linestyle='--', color='gray')

    plt.grid(axis='y', color='gray', alpha=0.25)
    plt.title(texto, fontsize=14)
    plt.ylabel('Inflação (%)',fontsize=12)
    plt.xlabel('Data', fontsize = 12)
    plt.legend(loc='best', fontsize='medium')

    plt.gca().xaxis.set_major_formatter(DateFormatter('%b/%y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 7]))     
    plt.gcf().autofmt_xdate()

    plt.xlim(df.index.min(), df.index.max())

    plt.gca().set_yticks(range(min, max + 1, 1))

    sns.despine()

    plt.show()

administrados = generate_plots(-5,20,df_administrados_ano_ma,'Administrados x Livres (MM3M) - Anualizada')
nucleos = generate_plots(0,12,df_nucleos_ano_ma,'Evolução dos núcleos (MM3M) - Anualizada')
categorias = generate_plots(-5,20,df_categorias_ano_ma,'Evolução da inflação (MM3M) - Anualizada')
comercializaveis = generate_plots(-2,15,df_comercializacao_ano_ma,'Comercialização - (MM3M) - Anualizada')

#### Portanto, os respectivos dataframes que serão utilizados são: 

##### df_componentes,df_componentes_ano, df_componentes_dessaz e df_componentes_ma 
##### df_categorias, df_componentes_ano, df_categorias_dessaz e df_categorias_ma
##### df_nucleos, df_nucleos_ano, df_nucleos_dessaz e df_nucleos_ma
##### df_comercializacao,df_comercializacao_ano df_comercializacao_dessaz e df_comercializacao_ma
##### df_administrados,df_administrados_ano, df_administrados_dessaz e df_administrados_ma

##### Cada critério possui quatro dataframes: Um que contém a variação mensal, um que contém a variação acumulada 12m, um dessazonalizado, e uma média móvel de 3 meses